<a href="https://colab.research.google.com/github/Nawapon19/NLP-Practice/blob/main/Intent_Recognition_using_TensorFlow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Intent Recognition using TensorFlow**

Intent recognition is a method of natural language processing, which deals with determining intent of a given sentence, or in simple terms “what the sentence means”. It is commonly used in chatbots, virtual assistants, and other conversational AI systems to understand user requests and provide appropriate responses.

In [27]:
# import necessary libraries and modules
import tensorflow as tf
import numpy as np
import random
import json

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

import warnings
warnings.filterwarnings('ignore')

In [28]:
# load dataset from json file
with open('Intent.json', 'r') as f:
  data = json.load(f)

# inspect dataset
print(data.keys()) # elements in json file
print(type(data['intents'])) # type of value in 'intents' dict
print(len(data['intents'])) # total number of examples
print(data['intents'][0].keys()) # elements in each example
data['intents'][-1] # print the last example

dict_keys(['intents'])
<class 'list'>
22
dict_keys(['intent', 'text', 'responses', 'extension', 'context', 'entityType', 'entities'])


{'intent': 'SelfAware',
 'text': ['Can you prove you are self-aware',
  'Can you prove you are self aware',
  'Can you prove you have a conscious',
  'Can you prove you are self-aware please',
  'Can you prove you are self aware please',
  'Can you prove you have a conscious please',
  'prove you have a conscious'],
 'responses': ['That is an interesting question, can you prove that you are?',
  'That is an difficult question, can you prove that you are?',
  'That depends, can you prove that you are?'],
 'extension': {'function': '', 'entities': False, 'responses': []},
 'context': {'in': '', 'out': '', 'clear': False},
 'entityType': 'NA',
 'entities': []}

**Data cleaning**

In [29]:
# create a function for data cleaning
# remove all non-alphabet characters
def clean(line):
  cleaned_line = ''
  for char in line:
    if char.isalpha():
      cleaned_line += char
    else:
      cleaned_line += ' '

  # rearrange the line by splitting each word and join them back together with ' '
  cleaned_line = ' '.join(cleaned_line.split())

  return cleaned_line

In [30]:
clean("I'm 27 years old now")

'I m years old now'

**Data Preprocessing**

In [31]:
# create list of intents
intents = []
unique_intents = []

# create list of all text data to create a corpus
text_input = []

# create dictionary for mapping intent with appropriate response
response_for_intent = {}

for intent in data['intents']:
  # update list of unique intents
  if intent['intent'] not in unique_intents:
    unique_intents.append(intent['intent'])
  # add cleaned text to the corpus
  for text in intent['text']:
    text_input.append(clean(text))
    # add intent of each text to lists of intent at the same index position
    intents.append(intent['intent'])
  # update intent in response dict
  if intent['intent'] not in response_for_intent:
    response_for_intent[intent['intent']] = []
  # update response for intent in response dict
  for response in intent['responses']:
    response_for_intent[intent['intent']].append(response)

In [32]:
print("Intent :", intents[0])
print("Number of Intent: ", len(intents))
print("Sample Input: ", text_input[0])
print("Length of text_input: ", len(text_input))
print("Sample Response: ", response_for_intent[intents[0]])

Intent : Greeting
Number of Intent:  143
Sample Input:  Hi
Length of text_input:  143
Sample Response:  ['Hi human, please tell me your GeniSys user', 'Hello human, please tell me your GeniSys user', 'Hola human, please tell me your GeniSys user']


**Tokenization and Embedding**

In [33]:
# create tokenizer object and fit the text input
# vectorize a text corpus,
# by turning each text into a sequence of integers (each integer being the index of a token in a dictionary)
tokenizer = Tokenizer(filters = '', oov_token = '<unk>') # <unk> unknown bucket for out of vocab words
tokenizer.fit_on_texts(text_input)

# tokenize text input and pad sequences to the same size
sequences = tokenizer.texts_to_sequences(text_input)
padded_sequences = pad_sequences(sequences, padding='pre')

print("Shape of Input Sequence: ", padded_sequences.shape)
padded_sequences[:5]

Shape of Input Sequence:  (143, 9)


array([[ 0,  0,  0,  0,  0,  0,  0,  0, 52],
       [ 0,  0,  0,  0,  0,  0,  0, 52, 53],
       [ 0,  0,  0,  0,  0,  0,  0,  0, 68],
       [ 0,  0,  0,  0,  0,  0,  0,  0, 39],
       [ 0,  0,  0,  0,  0,  0,  0, 39, 53]], dtype=int32)

In [34]:
sequences[:5]

[[52], [52, 53], [68], [39], [39, 53]]

In [35]:
text_input[:5]

['Hi', 'Hi there', 'Hola', 'Hello', 'Hello there']

In [36]:
print(text_input[-5:])
print(sequences[-5:])
print(padded_sequences[-5:])

['Can you prove you have a conscious', 'Can you prove you are self aware please', 'Can you prove you are self aware please', 'Can you prove you have a conscious please', 'prove you have a conscious']
[[11, 2, 28, 2, 66, 17, 67], [11, 2, 28, 2, 5, 50, 51, 14], [11, 2, 28, 2, 5, 50, 51, 14], [11, 2, 28, 2, 66, 17, 67, 14], [28, 2, 66, 17, 67]]
[[ 0  0 11  2 28  2 66 17 67]
 [ 0 11  2 28  2  5 50 51 14]
 [ 0 11  2 28  2  5 50 51 14]
 [ 0 11  2 28  2 66 17 67 14]
 [ 0  0  0  0 28  2 66 17 67]]


**Feature Extraction**

Neural network cannot process sentences, so numerical representation of sentences have to be provided to it, this is done by doing Feature Extraction, for that we map all words with their indexes and create a matrix mapping it to its category (intent).

In [37]:
intent_to_index = {}
categorical_target = []
index = 0

# create intent to index mapping
for intent in intents:
  if intent not in intent_to_index:
    intent_to_index[intent] = index
    index += 1
  # create a list of intents as index numbers
  categorical_target.append(intent_to_index[intent])

num_classes = len(intent_to_index)
print("Number of Intents: ", num_classes)

# convert intent_to_index to index_to_intent mapping
index_to_intent = {index: intent for intent, index in intent_to_index.items()}
index_to_intent

Number of Intents:  22


{0: 'Greeting',
 1: 'GreetingResponse',
 2: 'CourtesyGreeting',
 3: 'CourtesyGreetingResponse',
 4: 'CurrentHumanQuery',
 5: 'NameQuery',
 6: 'RealNameQuery',
 7: 'TimeQuery',
 8: 'Thanks',
 9: 'NotTalking2U',
 10: 'UnderstandQuery',
 11: 'Shutup',
 12: 'Swearing',
 13: 'GoodBye',
 14: 'CourtesyGoodBye',
 15: 'WhoAmI',
 16: 'Clever',
 17: 'Gossip',
 18: 'Jokes',
 19: 'PodBayDoor',
 20: 'PodBayDoorResponse',
 21: 'SelfAware'}

**One-Hot Encoding**

In [38]:
# apply one-hot encoding to categorical_target
categorical_vec = tf.keras.utils.to_categorical(categorical_target,
                                                num_classes = num_classes,
                                                dtype = 'int32')

print("Shape of Ca: ", categorical_vec.shape)
categorical_vec[:5]

Shape of Ca:  (143, 22)


array([[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]],
      dtype=int32)

**Model Building**

In [39]:
# set parameters for the neural network model
epochs = 100
embed_dim = 300
lstm_num = 50
output_dim = categorical_vec.shape[1]
input_dim = len(unique_intents)
print("Input Dimension: {}, \nOutput Dimension: {}".format(input_dim, output_dim))

Input Dimension: 22, 
Output Dimension: 22


A common model for intent recognition is the **recurrent neural network (RNN)** or its variant, the **long short-term memory (LSTM) network**. These networks can handle sequential data, such as sentences, effectively. We can also use pre-trained models like **BERT** or **GPT** to achieve better performance.

In [40]:
# RNN using sequential model,
# consist of embedding layer, lstm layer for sequence processing,
# and two dense layer for classification
model = tf.keras.models.Sequential([
    tf.keras.layers.Embedding(len(tokenizer.word_index) + 1, embed_dim),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(lstm_num, dropout = 0.1)),
    tf.keras.layers.Dense(lstm_num, activation = 'relu'),
    tf.keras.layers.Dropout(0.4),
    tf.keras.layers.Dense(output_dim, activation = 'softmax')
])

# set up optimizer for the model, 'adam' with learning rate = 0.001
optimizer = tf.keras.optimizers.Adam(lr = 0.001)

# compile the model
# use categorical crossentropy as loss function
model.compile(optimizer = optimizer, loss = 'categorical_crossentropy', metrics = ['accuracy'])

# print model summary
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, None, 300)         35700     
                                                                 
 bidirectional_2 (Bidirecti  (None, 100)               140400    
 onal)                                                           
                                                                 
 dense_4 (Dense)             (None, 50)                5050      
                                                                 
 dropout_2 (Dropout)         (None, 50)                0         
                                                                 
 dense_5 (Dense)             (None, 22)                1122      
                                                                 
Total params: 182272 (712.00 KB)
Trainable params: 182272 (712.00 KB)
Non-trainable params: 0 (0.00 Byte)
______________

**Training the model**

In [60]:
# train the model with labeled dataset
model.fit(padded_sequences, categorical_vec, epochs = epochs, verbose = 1)

Epoch 1/100
5/5 [==============================] - 0s 25ms/step - loss: 0.0030 - accuracy: 1.0000
Epoch 2/100
5/5 [==============================] - 0s 24ms/step - loss: 0.0097 - accuracy: 0.9930
Epoch 3/100
5/5 [==============================] - 0s 24ms/step - loss: 0.0020 - accuracy: 1.0000
Epoch 4/100
5/5 [==============================] - 0s 24ms/step - loss: 0.0044 - accuracy: 1.0000
Epoch 5/100
5/5 [==============================] - 0s 23ms/step - loss: 0.0127 - accuracy: 1.0000
Epoch 6/100
5/5 [==============================] - 0s 25ms/step - loss: 0.0044 - accuracy: 1.0000
Epoch 7/100
5/5 [==============================] - 0s 22ms/step - loss: 6.0095e-04 - accuracy: 1.0000
Epoch 8/100
5/5 [==============================] - 0s 29ms/step - loss: 0.0038 - accuracy: 1.0000
Epoch 9/100
5/5 [==============================] - 0s 24ms/step - loss: 0.0023 - accuracy: 1.0000
Epoch 10/100
5/5 [==============================] - 0s 27ms/step - loss: 0.0053 - accuracy: 1.0000
Epoch 11/100
5/

**Evaluate**

In [61]:
# create unseen text input with labels
# check if it works correctly
test_text_inputs = ["Hello",
                    "my name is adam",
                    "how are you?",
                    "can you guess my name?",
                    "Do you get me",
                    "Adios"]

test_intents = ["Greeting",
                 "GreetingResponse",
                 "CourtesyGreeting",
                 "CurrentHumanQuery",
                 "UnderstandQuery",
                 "GoodBye"]

# transform test data, tokenization and vectorization
test_sequences = tokenizer.texts_to_sequences(test_text_inputs)
test_padded_sequences = pad_sequences(test_sequences, padding = 'pre')
test_labels = np.array([unique_intents.index(intent) for intent in test_intents])
test_labels = tf.keras.utils.to_categorical(test_labels, num_classes = num_classes)

# evaluate the model
loss, accuracy = model.evaluate(test_padded_sequences, test_labels)

1/1 [==============================] - 0s 30ms/step - loss: 0.0118 - accuracy: 1.0000


**Predict**

In [62]:
# define a function response to predict intent and give appropriate response
def response(sentence):
  sent_tokens = []
  # split the input sentence into words
  words = sentence.split()
  # convert words to their corresponding indices
  for word in words:
    if word in tokenizer.word_index:
      sent_tokens.append(tokenizer.word_index[word])
    else:
      # hanedle unknown words]
      sent_tokens.append(tokenizer.word_index['<unk>'])

  # tf.expand_dims -> given a tensor input, this operation inserts a dimension of
  # length 1 at the dimension index 'axis' of input's shape.
  sent_tokens = tf.expand_dims(sent_tokens, 0)

  # predict numerical category (probs)
  pred = model(sent_tokens)

  # category to intent (index number)
  pred_class = np.argmax(pred.numpy(), axis = 1)

  # return random response to the intent and intent type
  return random.choice(response_for_intent[index_to_intent[pred_class[0]]]), index_to_intent[pred_class[0]]

**Chatbots: Intent Recognition**

In [63]:
# create a loop to take input from user
# recognize intent and give appropriate response
print("Note: Enter 'quit' to break the loop.")
while True:
  query = input('You: ')
  if query.lower() == 'quit':
    break
  bot_response, typ = response(query)
  print("Bot: {} -- TYPE: {}".format(bot_response, typ))
  print()

Note: Enter 'quit' to break the loop.
You: Hi, Who are you?
Bot: Hello, I am great, how are you? Please tell me your GeniSys user -- TYPE: CourtesyGreeting

You: can you prove you have a concious?
Bot: That depends, can you prove that you are? -- TYPE: SelfAware

You: Can you prove you are self-aware?
Bot: That is an interesting question, can you prove that you are? -- TYPE: SelfAware

You: My name is Nawapon Srikrajang.
Bot: OK! hi <HUMAN>, what can I do for you? -- TYPE: GreetingResponse

You: Tell me a joke
Bot: What do you call cheese that isn't yours?  Nacho cheese. -- TYPE: Jokes

You: Tell me ajokes
Bot: Let me see -- TYPE: WhoAmI

You: Tell me a Jokes
Bot: 'Where are you going on holiday?' John asked Trevor. 'We're off to Thailand this year', Trevor replied. 'Oh; aren't you worried that the very hot weather might disagree with your wife?' asked John. 'It wouldn't dare', said Trevor. -- TYPE: Jokes

You: quit
